# Administrating Portal and Server

## Managing Online and Portal


<img src="./img/portal.png" width=50%/>

- ArcGIS Online (AGOL)
- Portal for ArcGIS

### Administration Overview

<img src="./img/gis.admin.png"/>

### Portal vs AGOL

<h3>Organizational differences</h3>

<table border="1" class="dataframe"><thead><tr style="text-align: right;"><th>Function</th><th>ArcGIS Online</th><th>ArcGIS Enterprise</th></tr></thead><tbody><tr><td>collaborations</td><td>X</td><td>X</td></tr><tr><td>credits</td><td>X</td><td></td></tr><tr><td>federation</td><td></td><td>X</td></tr><tr><td>license</td><td>X</td><td>X</td></tr><tr><td>logs</td><td></td><td>X</td></tr><tr><td>machines</td><td></td><td>X</td></tr><tr><td>metadata</td><td>X</td><td>X</td></tr><tr><td>password_policy</td><td>X</td><td>X</td></tr><tr><td>security</td><td></td><td>X</td></tr><tr><td>server</td><td></td><td>X</td></tr><tr><td>site</td><td></td><td>X</td></tr><tr><td>system</td><td></td><td>X</td></tr><tr><td>ux</td><td>X</td><td>X</td></tr></tbody></table>

Most properties on ArcGIS Online are available on ArcGIS Enterprise except 'credit reporting' because ArcGIS Enterprise does not consume credits.

<blockquote><b>Note:</b> You need to log in using a named user account with administrator privileges. When you login, the API detects if you are an organizational administrator, then, the <b>GIS</b> object will ensure you gain access to the <b>admin</b> module.</blockquote>


In [ ]:
from arcgis.gis import GIS
gis_agol = GIS(profile="adminProfile")

### Licensing 

- Supports assigning licenses for Esri premium apps
- The licenses include: 
    + ArcGIS Pro, Navigator for ArcGIS, AppStudio for ArcGIS Standard, Drone2Map for ArcGIS, ArcGIS Business Analyst web app, ArcGIS Community Analyst, GeoPlanner for ArcGIS


**Listing Licenses**

In [ ]:
license = gis_agol.admin.license
license.all()

**Getting a Single License**

In [ ]:
pro_license = license.get('ArcGIS Pro')
pro_license

**Viewing Licensing Reports**

- provides both visual and tabular reports

**Tabular Reports**

In [ ]:
report = pro_license.report
report

**Revoking Entitlements**

- `revoke` removes a license for the product assigned to a user.

In [ ]:
pro_license..revoke(username="entitleduser", entitlements="*")

In [ ]:

report[report['Entitlement'] == 'desktopStdN']


#### Releasing ArcGIS Pro License for Offline Users

- Users can check out licenses, but it holds a license from anyone else to use
- Administrators can force the release of that license on enterprise

```python
licenses.release_license('username')
{'status' : 'success'}
```

### Credit Management

- The currency of ArcGIS Online
- Allows users to do analysis, enrich data, and much more
- Hosting services cost credit as well.

**Access the CreditManager**

In [ ]:
cm = gis_agol.admin.credits
cm

**View Available Credits**

- Get the total number of credits on an Organization

In [ ]:
cm.credits

**Managing Credits**

- Setup budgeting rules by enabling management

In [ ]:
if cm.is_enabled:
    #print(cm.enable())
cm.is_enabled

**Checking the Default Limit**

- The default credit setting is unlimited (-1)


In [ ]:
cm.default_limit

**Demo: Setting the default user credits**

In [ ]:
# %load ./solutions/setting_credits.py
if cm.default_limit == -1:
    cm.default_limit = 500
cm.default_limit

**Allocating Credits to a User**

- Assignment of credits beyond the default is sometimes necessary

In [ ]:
cm.allocate(username=gis_agol.users.me.username, 
            credits=300)

**Give Specific User Unlimited Credits**

In [ ]:
#cm.allocate(username=gis_agol.users.me.username,credits=-1)

**Disabling Credit Management**

- `disable()` on credit manager will disable any credit monitoring
- **THIS IS A BAD IDEA**

### Metadata Management

- Information about information
- Allows users to provide robust information about datasets beyond the description, title, tags, etc...

### User History

- Examine what users are doing with the organization
- Ensure specific content does not change or get modified


In [ ]:
%matplotlib inline
import datetime
import pandas as pd
then = datetime.datetime.now() - datetime.timedelta(days=4)
df = pd.read_csv(gis_agol.admin.history(start_date=then, num=100000))
df.action.value_counts().plot.bar()

### Portal Logs

- A record of events that occurred
- Used for monitoring and troubleshooting portal

#### Examples of Logs Incidents:

+ Installation and upgrade events, such as authorizing the software and creating the portal website
+ Publishing of services and items, such as hosted services, web maps, and data items
+ Content management events, such as sharing items, changing item ownership, and adding, updating, moving, and deleting items
+ Security events, such as users logging in to the portal, creating, deleting, and disabling users, creating and changing user roles, updating HTTP and HTTPS settings, import and export of security certificates, and updating the portal's identity store
+ Organization management events, such as adding and configuring groups, adding or removing users from a group, configuration of the gallery, basemaps, utility services, and federated servers, and configuring log settings and deleting logs
+ General events, such as updating the portal's search index and restarting the portal

In [ ]:
from arcgis.gis import GIS
gis = GIS(profile="your_enterprise_profile")
logs = gis.admin.logs
logs

#### Log Settings

- Modify, update basic storage and save setting

In [ ]:
logs.settings

#### Query Portal Logs

In [ ]:
import datetime
import pandas as pd
results = logs.query(start_time=datetime.datetime.now() - datetime.timedelta(days=10))
results

In [ ]:
%matplotlib inline
df = pd.DataFrame(results['logMessages'])
df.type.value_counts().plot.bar()

In [ ]:
df.head()

### Deep Dive into PortalAdmin API

####  Inspecting the Portal's Machines

- query the machines that power your portal



In [ ]:
machines = gis.admin.machines
machines

In [ ]:
machines.list()

**Check if Machine is Running**

In [ ]:
machine = machines.list()[0]
machine.status()

#### System Directories

- inspect the physical pathes where resources are stored

In [ ]:
directories = gis.admin.system.directories
directories

In [ ]:
d = directories[0]
d.properties

#### The System 

In [ ]:
system = gis.admin.system
system

In [ ]:
system.properties

#### Re-indexing

- Sometimes artifacts remain after deleting items
- Forcing re-indexing can solved that problem

In [ ]:
system.index_status

In [ ]:
#system.reindex(mode="SEARCH_MODE")

In [ ]:
system.index_status

## Managing ArcGIS Server

### Managing Federated Servers

- The `admin` property provides useful tools to manage ArcGIS Server instances

In [ ]:
gis = GIS(profile="enterpriseAdmin")
servers = gis.admin.servers
servers

#### Listing the Federated Servers


In [ ]:
s = servers.list()
s

#### Check if Servers are Working

- validate ensures everything is federated and running correctly

In [ ]:
servers.validate()

### Connecting to a Server

#### Accessing Single Server


In [ ]:
server = s[0]
server

In [ ]:
server.properties

#### Accessing Server Logs

- Like the portal Logs, server provide a host of information

In [ ]:
logs = server.logs
logs

In [ ]:
logs.settings

### Managing service folders

**Creating a Folder** 

- use `create_folder`

In [ ]:
#server.services.create_folder("crime_analysis")

**Delete a Folder** 

- use `delete_folder`

In [ ]:
#server.services.delete_folder('crime_analysis')

### Managing Services

- Access service management from `services` property
- Provides the ability start,stop, delete, and modify services

In [ ]:
services = server.services
services

#### Checking if Service Exists

To check if a service exists on your server, call the `exists` method and specify the folder name, service name and type. You can also use this method to verify if a folder exists on the server.

In [ ]:
services.exists(folder_name='Hosted', name='Ports', service_type='FeatureServer')

#### Demo: Listing all Services

In [ ]:
for folder in services.folders:
    for s in services.list(folder):
        print(s)

#### Control a Service's State

- `start`, `stop` and `restart` services

In [ ]:
for service in services.list():
    if service.properties.serviceName == 'SampleWorldCities':
        break
service

**Check the Service Status**

- Shows if the services is running or not

In [ ]:
service.status

In [ ]:
#service.stop()

In [ ]:
service.status

In [ ]:
#service.start()

In [ ]:
service.status

### Modifying a Service

- modify extensions, pooling, etc...

In [ ]:
for service in services.list():
    if service.properties.serviceName == 'SampleWorldCities':
        break
service

In [ ]:
for ext in service.extensions:
    if ext.typeName == "KmlServer":
        ext.enabled = True
[(ext.typeName, ext.enabled) for ext in service.extensions]

### Publishing an SD File

- directly publish SD file to server


In [ ]:
fp = r"./data/dino_AttachmentManager_basic.sd"
services.publish_sd(sd_file=fp)

In [ ]:
s = services.list()[0]
s.properties

In [ ]:
s.delete()

### Server Logs

ArcGIS Server records events that occur, and any errors associated with those events, to logs. Logs are an important tool for monitoring and troubleshooting problems with your site. Information in the logs will help you identify errors and provide context on how to address problems

In [ ]:
logs = server.logs
logs

In [ ]:
logs.settings

#### Filtering and querying server logs

In [ ]:
import datetime
import pandas as pd
now = datetime.datetime.now()
start_time = now - datetime.timedelta(days=10)
start_time

In [ ]:
recent_logs = logs.query(start_time = start_time)

#print a message as a sample
recent_logs['logMessages']

### Monitoring Server Usage

ArcGIS Server records various service statistics, such as total requests, average response time and timeouts. Administrators and publishers can use this information to monitor service activity to better understand how clients are using services. For example, monitoring server statistics help you answer questions such as:

- What is the total number of requests that my ArcGIS Server site handled during the past week?
- How was the service request load distributed during the past month?
- How are my services performing on an hourly basis?
- What was the maximum number of service instances used at any given time for a particular service?

In [ ]:
usage = server.usage
usage

#### Using built-in report

In [ ]:
reports = usage.list()
reports

In [ ]:
for r in reports:
    print(r.properties['reportname'])

#### Querying maximum response times for the last 7 days

In [ ]:
data = reports[0].query()

In [ ]:
from datetime import datetime
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
%matplotlib inline

In [ ]:
#store reponse times in Y axis
data_y = data['report']['report-data'][0][0]['data']

#convert dates to readable dates and store in X axis
data_x = [pd.to_datetime(datetime.fromtimestamp(d//1000)) \
          for d in data['report']['time-slices']]

df = pd.DataFrame(list(zip(data_x, data_y)), columns=["date", "count"])
q = df['count'].isnull() # change NaN values to 0
df.loc[q, 'count'] = 0
df.index = df['date']
df['count'] = df['count'] 

ax = df['count'].plot(kind='bar', x=df['date'])
ticklabels = ['']*len(df.index)
ticklabels[::4] = [item.strftime('%b %d') for item in df.index[::4]]
ax.xaxis.set_major_formatter(ticker.FixedFormatter(ticklabels))
ax.set_title('Maximum reponse time in the last 7 days')
ax.set_ylabel('Time in seconds')
plt.gcf().autofmt_xdate()
plt.show()

#### Creating Quick Reports

- On the fly reporting
- Data is not saved

**Metrics Available**

- RequestCount - the number of requests received
- RequestsFailed - the number of requests that failed
- RequestsTimedOut - the number of requests that timed out
- RequestMaxResponseTime - the maximum response time
- RequestAvgResponseTime - the average response time
- ServiceActiveInstances - the maximum number of active (running) service instances sampled at 1 minute intervals, for a specified service


In [ ]:
data = usage.quick_report(since="LAST_MONTH", metrics="RequestCount")
data.keys()

In [ ]:
type(data['report']['report-data']), len(data['report']['time-slices'])

In [ ]:
import pandas as pd
data_flat = {
    #'report_data' : data['report']['report-data'],
    'time_slices' : data['report']['time-slices']
}
for d in data['report']['report-data'][0]:
    data_flat[d['metric-type']] = d['data']

In [ ]:
pd.DataFrame(data_flat).tail()

In [ ]:
#store reponse times in Y axis
data_y = data_flat["RequestCount"]

#convert dates to readable dates and store in X axis
data_x = [pd.to_datetime(datetime.fromtimestamp(d//1000)) \
          for d in data_flat['time_slices']]

df = pd.DataFrame(list(zip(data_x, data_y)), columns=["date", "count"])
q = df['count'].isnull() # change NaN values to 0
df.loc[q, 'count'] = 0
df.index = df['date']
df['count'] = df['count'] 

ax = df['count'].plot(kind='bar', x=df['date'])
ticklabels = ['']*len(df.index)
ticklabels[::4] = [item.strftime('%b %d') for item in df.index[::4]]
ax.xaxis.set_major_formatter(ticker.FixedFormatter(ticklabels))
ax.set_title('Requests over the last 30 days')
ax.set_ylabel('Number of requests')
plt.gcf().autofmt_xdate()
plt.show()

## Questions?